In [1]:
import pandas as pd
from gensim.models.fasttext import FastText 
import logging

df = pd.read_csv('lenta_tech.csv')

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [37]:
def word_averaging(wv, words):
    mean = np.zeros((wv.vector_size,))
    
    for word in words:
        mean += wv.get_vector(word)

    mean = gensim.matutils.unitvec(mean)
    return mean

def word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, review) for review in tqdm(text_list)])

In [3]:
def normalize_text(ser: pd.Series):
    return ser.str.lower().str.replace(r'[\W_]', ' ')

df.text = normalize_text(df.text)

/home/gorodion/go_venv/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [4]:
df.text = df.text.apply(lambda x: [i for i in x.split() if len(i) > 1])

In [5]:
model = FastText(
    df.text, 
    vector_size=200, 
    window=35, 
    min_count=3, 
    workers=32, 
    epochs=5, 
    sg=1, 
    bucket=400_000
)

2022-06-10 16:41:11,863 : INFO : collecting all words and their counts
2022-06-10 16:41:11,864 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-06-10 16:41:12,434 : INFO : PROGRESS: at sentence #10000, processed 1781619 words, keeping 136469 word types
2022-06-10 16:41:12,938 : INFO : collected 190157 word types from a corpus of 3252866 raw words and 18832 sentences
2022-06-10 16:41:12,939 : INFO : Creating a fresh vocabulary
2022-06-10 16:41:13,345 : INFO : FastText lifecycle event {'msg': 'effective_min_count=3 retains 78657 unique words (41.36424112706869%% of original 190157, drops 111500)', 'datetime': '2022-06-10T16:41:13.344438', 'gensim': '4.1.2', 'python': '3.7.3 (tags/debian/3.7.3-1+xenial1-dirty:e8c58d1, Jun 20 2019, 08:00:00) \n[GCC 5.4.0 20160609]', 'platform': 'Linux-5.4.111-14-x86_64-with-Ubuntu-16.04-xenial', 'event': 'prepare_vocab'}
2022-06-10 16:41:13,345 : INFO : FastText lifecycle event {'msg': 'effective_min_count=3 leaves 3112383 wo

In [7]:
mkdir ft

In [8]:
model.save('ft/ft.model')

2022-06-10 17:11:45,813 : INFO : FastText lifecycle event {'fname_or_handle': 'ft/ft.model', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2022-06-10T17:11:45.813146', 'gensim': '4.1.2', 'python': '3.7.3 (tags/debian/3.7.3-1+xenial1-dirty:e8c58d1, Jun 20 2019, 08:00:00) \n[GCC 5.4.0 20160609]', 'platform': 'Linux-5.4.111-14-x86_64-with-Ubuntu-16.04-xenial', 'event': 'saving'}
2022-06-10 17:11:45,815 : INFO : not storing attribute vectors
2022-06-10 17:11:45,816 : INFO : storing np array 'vectors_vocab' to ft/ft.model.wv.vectors_vocab.npy
2022-06-10 17:11:45,884 : INFO : storing np array 'vectors_ngrams' to ft/ft.model.wv.vectors_ngrams.npy
2022-06-10 17:11:46,126 : INFO : not storing attribute buckets_word
2022-06-10 17:11:46,127 : INFO : storing np array 'syn1neg' to ft/ft.model.syn1neg.npy
2022-06-10 17:11:46,175 : INFO : not storing attribute cum_table
2022-06-10 17:11:46,245 : INFO : saved ft/ft.model


embeds

In [10]:
model

In [12]:
df = pd.read_csv('lenta_tech.csv')

In [13]:
df.text = normalize_text(df.text)
df.text = df.text.apply(lambda x: [i for i in x.split() if len(i) > 1])

/home/gorodion/go_venv/lib/python3.7/site-packages/pyforest/__init__.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  from ._imports import *


In [38]:
embeds = word_averaging_list(model.wv, df.text)

  0%|          | 0/18832 [00:00<?, ?it/s]

In [40]:
np.save('ft_embeds.npy', embeds)

gen_embeds

In [41]:
generated = pd.read_csv('generated.txt', header=None, squeeze=True)

In [42]:
generated = normalize_text(generated)
generated = generated.apply(lambda x: [i for i in x.split() if len(i) > 1])

/home/gorodion/go_venv/lib/python3.7/site-packages/pyforest/__init__.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  from ._imports import *


In [43]:
gen_embeds = word_averaging_list(model.wv, generated)

  0%|          | 0/2000 [00:00<?, ?it/s]

In [44]:
np.save('ft_gen_embeds.npy', embeds)